# More bike tripping

It makes symmetrical sense to point bikes which started the day at a station against bikes that ended the day there. This is that latter half.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
import io
import zipfile
from tqdm import tqdm

In [2]:
stations = pd.read_csv("../data/final/june_22_station_metadata.csv", index_col=0)

In [3]:
june_22 = pd.read_csv("../data/final/all_june_22_citibike_trips.csv", index_col=0)

In [4]:
june_22.head(2)

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
1004842,25585,1978.0,450,40.762272,-73.987882,W 49 St & 8 Ave,1,334,40.742388,-73.997262,W 20 St & 7 Ave,6/22/2016 00:00:11,6/22/2016 00:13:17,785,Subscriber
1004843,23365,1997.0,486,40.746201,-73.988557,Broadway & W 29 St,1,529,40.757570,-73.990985,W 42 St & 8 Ave,6/22/2016 00:00:29,6/22/2016 00:08:00,450,Subscriber


In [10]:
june_22[june_22['bikeid'] == 25585].sort_values(by='starttime', ascending=False).tail(1)['starttime']

1025    2016-22-06 23:11:01
Name: starttime, dtype: object

In [11]:
june_22[june_22['bikeid'] == 25585].sort_values(by='starttime', ascending=False).head(1)['starttime']

1060061    6/22/2016 23:24:15
Name: starttime, dtype: object

In [ ]:
station_bikesets = {station_id: [] for station_id in stations.index}

for bike_id in tqdm(np.unique(june_22['bikeid'])):
    first_trip_station = june_22[june_22['bikeid'] == bike_id].sort_values(by='starttime', ascending=False).iloc[0]['start station id']
    station_bikesets[first_trip_station].append(bike_id)